In [126]:
def averaging():
    first_onset_trow = []
    second_onset_trows = []
    third_onset_trows = []
    onset_time = [0, 0.5, 1]
    tseries = read_timeseries_data(subnum="01")
    for i in onset_time:
        TR = (i/1.5) + 3
        TR = str(TR)
        if TR[2:3] == "0":
            TR2 = int(float(TR))
            trow = tseries.iloc[TR2]
            first_onset_trow.append(trow)
            df = pd.concat(first_onset_trow, axis=1, ignore_index=False)
            df = (df.transpose())
        elif TR[2:3] == "3":
            TR_1 = int(float(TR))
            TR_1 = math.ceil(TR_1)
            trow = tseries.iloc[TR_1]
            second_onset_trows.append(trow)
            df1 = pd.concat(second_onset_trows, axis=1, ignore_index=False).transpose()
            TR_2 = TR_1 +1
            trow2 = tseries.iloc[TR_2]
            second_onset_trows.append(trow2)
            df2 = pd.concat(second_onset_trows, axis=1, ignore_index=False).transpose() 
            df2.insert(loc=1, column='Weights', value=[2,1])
            column_headers = list(df2.columns.values)
            weights = []
            for x in column_headers:
                weighted_average = np.average(a=df2[f'{x}'], weights=df2['Weights'])
                weights.append(weighted_average)
            res = {}
            for col in column_headers:
                for value in weights:
                    res[col] = value
                    weights.remove(value)
                    break
            df3 = pd.DataFrame.from_dict(res, orient='index').transpose()
            df3 = df3.drop('Weights', axis=1)
        elif TR[2:3] == "6":
            TR_3 = int(float(TR))
            TR_3 = math.ceil(TR_3)
            TR_4 = TR_3 + 1
            trow = tseries.iloc[TR_1]
            third_onset_trows.append(trow)
            df1_ = pd.concat(third_onset_trows, axis=1, ignore_index=False).transpose()
            TR_2 = TR_1 +1
            trow2 = tseries.iloc[TR_2]
            third_onset_trows.append(trow2)
            df2_ = pd.concat(third_onset_trows, axis=1, ignore_index=False).transpose() 
            df2_.insert(loc=1, column='Weights', value=[1,2])
            column_headers = list(df2_.columns.values)
            weights = []
            for x in column_headers:
                weighted_average = np.average(a=df2_[f'{x}'], weights=df2_['Weights'])
                weights.append(weighted_average)
            res = {}
            for col in column_headers:
                for value in weights:
                    res[col] = value
                    weights.remove(value)
                    break
            df3_ = pd.DataFrame.from_dict(res, orient='index').transpose()
            df3_ = df3_.drop('Weights', axis=1)
    df6 = pd.concat([df, df3], ignore_index=True, axis=0)
    final_df = pd.concat([df6, df3_], ignore_index=True, axis=0)
    return final_df
        
averaging()        

,7Networks_LH_Vis_1,7Networks_LH_Vis_2,7Networks_LH_Vis_3,7Networks_LH_Vis_4,7Networks_LH_Vis_5,7Networks_LH_Vis_6,7Networks_LH_Vis_7,7Networks_LH_Vis_8,7Networks_LH_Vis_9,7Networks_LH_Vis_10,...,7Networks_RH_Default_pCunPCC_11,7Networks_RH_Default_pCunPCC_12,7Networks_RH_Default_pCunPCC_13,7Networks_RH_Default_pCunPCC_14,7Networks_RH_Default_pCunPCC_15,7Networks_RH_Default_pCunPCC_16,7Networks_RH_Default_pCunPCC_17,7Networks_RH_Default_pCunPCC_18,7Networks_RH_Cont_pCun_2,7Networks_RH_Cont_pCun_4
0,-0.668825,-0.508630,-0.357809,-0.348872,0.801699,-0.025604,-0.038348,-0.281745,-0.416146,0.220948,...,-0.046802,0.299463,0.236304,-0.127338,0.099261,-0.028504,0.367355,-0.531135,0.253888,0.431395
1,-0.804250,-0.676582,-0.129873,-0.557622,0.699195,-0.345469,-0.106851,-0.408889,-0.586189,-0.154285,...,-0.237012,-0.034760,-0.069792,-0.608112,-0.290366,-0.405901,0.212785,-0.543626,0.054429,0.157987
2,-0.939676,-0.844534,0.098063,-0.766372,0.596691,-0.665334,-0.175354,-0.536033,-0.756232,-0.529518,...,-0.427222,-0.368984,-0.375887,-1.088886,-0.679994,-0.783298,0.058216,-0.556117,-0.145029,-0.115421


In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
import math

In [47]:
def read_timeseries_data(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-{runnum}_space-fsLR_den-91k_bold_timeseries.tsv"
    data = pd.read_csv(fname, sep="\t")
    return data

def read_metadata(subnum, dataloc="../tseries", runtype="pair", runnum=1):
    fname = f"{dataloc}/sub-{subnum}_task-{runtype}_run-0{runnum}_events.tsv"
    dat = pd.read_csv(fname, sep="\t")
    return dat

def reducing_metadata(image_of_interest, subnum="01"):
    meta = read_metadata(subnum)
    img_rows = meta[meta['trial_type'] == image_of_interest]
    onset_column = img_rows["onset"].to_list()
    print(onset_column)
    int_list = []
    for i in onset_column:
        TR = (i/1.5) + 3
        TR = int(math.ceil(TR))
        int_list.append(TR)
    #return int_list
    print(int_list)

def pull_timeseries(imagepair,subnum="01"):
    int_list = reducing_metadata(imagepair)
    tseries = read_timeseries_data(subnum)
    all_trows = []
    for i in int_list:
        trow = tseries.iloc[i]
        all_trows.append(trow)
    df = pd.concat(all_trows, axis=1, ignore_index=False)
    df = (df.transpose())
    display(df)

def find_image_pairs(subnum="01"):
    meta = read_metadata(subnum)
    image_pairs = meta["trial_type"].unique()
    return image_pairs
    
def main():
    image = find_image_pairs()
    #pull_timeseries(image[0])
    reducing_metadata(image_of_interest = image[0])

main()
    
    
    

[6.0, 104.0, 122.0, 206.0, 288.0]
[7, 73, 85, 141, 195]


1. For a given image, reduce the meta dataframe to only rows that correspond to that image
2. Pull the onset column from these rows (should be left with 5 numbers)
3. Divide all onsets by 1.5 (gets it into TR units), then add 3 (for hemodynamic lag)
4. Round these (this may have to be conditional), to get just 5 integers
5. Use those numbers as indices into the timeseries dataframe - should be left with 5 rows of data.
6. Scale to function that does this for a given subject, runtype, run number and trial type.